In [1]:
import numpy as np
import importlib, os, datetime, pickle
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from quick_sim import setup_sim


# import edward_tools.fq_runner as fq_runner
from edward_tools.cfqp_3D_potential import coupled_flux_qubit_2p5D_non_linear_approx_pot, coupled_flux_qubit_2p5D_non_linear_approx_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt

from edward_tools import cfq_3D_runner, coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization


# create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
# coupled_fq_runner = importlib.reload(cfq_runner)
# coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
# create_system = coupled_fq_protocol_library.create_system
# get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
# plotCutlines = coupled_fq_protocol_library.plotCutlines

In [2]:
{
    0: "U0_1", 1: "U0_2", 2: "U0_3", 3: "gamma_1", 4: "gamma_2", 5: "gamma_3", 6: "beta_1", 7: "beta_2",
    8: "beta_3", 9: "d_beta_1", 10: "d_beta_2", 11: "d_beta_3", 12: "phi_1ax", 13: "phi_1bx",
    14: "phi_2x", 15: "phi_3x", 16: "phi_1xdc", 17: "phi_2xdc", 18: "phi_3xdc", 19: "mu_12", 20: "mu_13"
}

default_potential_params = [1, 1, 1, 20, 20, 20, 2.3, 2.3, 2.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [3]:
len("""U0_1, U0_2, U0_3, gamma_1, gamma_2, gamma_3, beta_1, beta_2, beta_3, d_beta_1, d_beta_2, d_beta_3, phi_1ax, phi_1bx, phi_2x, phi_3x, phi_1xdc, phi_2xdc, phi_3xdc, mu_12, mu_13""".split(", "))

21

In [4]:
default_potential_params

[1, 1, 1, 20, 20, 20, 2.3, 2.3, 2.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [5]:
len(default_potential_params)

21

In [6]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.3
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 1
I_m_factor = 0

I_p_1, I_p_2, I_p_3 = 5e-6, 5e-6 , 5e-6  # Amp
I_m_1, I_m_2, I_m_3 = 7e-9 * I_m_factor, 7e-9 * I_m_factor, 7e-9 * I_m_factor              # Amp
R_1, R_2, R_3 = 1 * R_factor, 1 * R_factor, 1 * R_factor           # ohm
C_1, C_2, C_3 = 500e-15 * C_factor, 500e-15 * C_factor, 500e-15 * C_factor                 # F
L_1, L_2, L_3 = 140e-12 * L_factor, 140e-12 * L_factor, 140e-12 * L_factor                 # H  

freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)

In [7]:
m_c = C_1
m_1, m_2, m_3 = C_1, C_2, C_3
x_c = PHI_0 / (2 * np.pi)
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

# phi
lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_3 * R_3)
theta_3  = 1 / (C_3/C_1)
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_3 * C_3**2))

# phi_dc

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_4  = 4
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_5 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_5  = 4 / (C_2/C_1)
eta_5    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_6 = 2 * np.sqrt(L_1 * C_1) / (C_3 * R_3)
theta_6  = 4 / (C_3/C_1)
eta_6    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_3 * C_3**2))

gamma_1, gamma_2, gamma_3 = 20, 20, 20


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;
beta_3 = 2 * np.pi * L_3 * I_p_3 / PHI_0;

beta_1 = 2.3
beta_2 = 2.3
beta_3 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;
d_beta_3 = 2 * np.pi * L_3 * I_m_3 / PHI_0;

_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4, lambda_5, lambda_6])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4, theta_5, theta_6])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4, eta_5, eta_6])

v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/m_3)) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_5 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c
v_6 = np.random.normal(0, np.sqrt(k_BT/(m_3/4))) / v_c

In [8]:
print(f"L_1 = {L_1 * 1e12:.3g}pH, T = {T}K")
print(f"freq = {freq / 1e9:.3g}GHz")
print(characteristic_time)
print(1/U0_1)

L_1 = 140pH, T = 4.3K
freq = 120GHz
8.366600265340756e-12
0.07670153130674186


In [9]:
experiment_comment = "Testing"

In [10]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 500
params['dt'] = 1/5000
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = experiment_comment
params['capacitance'] = np.array([C_1, C_2, C_3, C_1/4, C_2/4, C_3/4])
params['mass'] = np.array([1, 1, 1, 1/4, 1/4, 1/4])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::10] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [11]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-4, -4, -4]), np.array([4, 4, 4])]



# µ = 0.06, φ2xdc = 1.79, φ1x = 0.59, and φ2x = 0.10.
    
initial_parameter_dict = {
    "U0_1": U0_1, "U0_2": U0_2, "U0_3": U0_2, "gamma_1": gamma_1, "gamma_2": gamma_2, "gamma_3": gamma_3, 
    "beta_1": beta_1, "beta_2": beta_2, "beta_3": beta_3, "d_beta_1": d_beta_1, "d_beta_2": d_beta_2, "d_beta_3": d_beta_3, 
    "phi_1ax": 0, "phi_1bx": 0, "phi_2x": 0, "phi_3x": 0, "phi_1xdc": 0, "phi_2xdc": 0, "phi_3xdc": 0, "mu_12":0, "mu_13": 0
}


In [12]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 1, 20, 20, 20, 2.3, 2.3, 2.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[phi_1_bound, phi_2_bound, phi_3_bound, phi_1dc_bound, phi_2dc_bound, phi_3dc_bound] = np.array([4, 4, 4, 4, 4, 4])
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_3_bound, -phi_1dc_bound, -phi_2dc_bound, -phi_3dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_3_bound, phi_1dc_bound, phi_2dc_bound, phi_3dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_2p5D_non_linear_approx_pot, coupled_flux_qubit_2p5D_non_linear_approx_force, 21, 6,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)


# protocol settings

In [13]:
zeroDissipation = False
saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "R_factor": R_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_p_3": I_p_3, "I_m_1": I_m_1, "I_m_2": I_m_2, "I_m_3": I_m_3,
    "R_1": R_1, "R_2": R_2, "R_3": R_3, "C_1": C_1, "C_2": C_2, "C_3": C_3, "L_1": L_1, "L_2": L_2, "L_3": L_3, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor), "gamma": gamma_1
}


# bookmark
initial_parameter_dict["phi_1xdc"] = 0
initial_parameter_dict["phi_2xdc"] = 0
initial_parameter_dict["phi_3xdc"] = 0
initial_parameter_dict["phi_1ax"]   = 0
initial_parameter_dict["phi_1bx"]   = 0
initial_parameter_dict["phi_2x"]   = 0
initial_parameter_dict["phi_3x"]   = 0
initial_parameter_dict["mu_12"]      = 0
initial_parameter_dict["mu_13"]      = 0

In [14]:
protocol_list = [
{
    'duration': 100,
    'phi_1ax': 0.0,
    'phi_1bx': 0.0,
    'phi_2x': 0.0,
    'phi_3x': 0.0,
    'mu_12': 0.0,
    'mu_13': 0.0,
    'phi_1xdc': 0,
    'phi_2xdc': 0,
    'phi_3xdc': np.pi,
    'name': 'conditional_tilt_xz'
},
{
    'duration': 100,
    'phi_1ax': 0.0,
    'phi_1bx': 0.0,
    'phi_2x': 0.0,
    'phi_3x': 0.0,
    'mu_12': 0.0,
    'mu_13': 0.0,
    'phi_1xdc': 0,
    'phi_2xdc': 0,
    'phi_3xdc': np.pi,
    'name': 'conditional_tilt_xz'
},
# {
#     'duration': 100,
#     'phi_1ax': 0.0,
#     'phi_1bx': 0.0,
#     'phi_2x': 0.0,
#     'phi_3x': -1.0,
#     'mu_12': 0.0,
#     'mu_13': 0.0,
#     'phi_1xdc': 0,
#     'phi_2xdc': 0,
#     'phi_3xdc': np.pi,
#     'name': 'conditional_tilt_xz'
# },
# {
#     'duration': 100,
#     'phi_1ax': 0.0,
#     'phi_1bx': 0.0,
#     'phi_2x': 0.0,
#     'phi_3x': 0.0,
#     'mu_12': 0.0,
#     'mu_13': 0.0,
#     'phi_1xdc': 0,
#     'phi_2xdc': 0,
#     'phi_3xdc': 0,
#     'name': 'conditional_tilt_xz'
# },
# {
#     'duration': 100,
#     'phi_1ax': 0.0,
#     'phi_1bx': 0.0,
#     'phi_2x': 0.0,
#     'phi_3x': 0.0,
#     'mu_12': 0.4,
#     'mu_13': 0.4,
#     'phi_1xdc': 0,
#     'phi_2xdc': 0,
#     'phi_3xdc': 0,
#     'name': 'conditional_tilt_xz'
# },
# {
#     'duration': 300,
#     'phi_1ax': 0.0,
#     'phi_1bx': 0.0,
#     'phi_2x': 0.0,
#     'phi_3x': 0.0,
#     'mu_12': 0.4,
#     'mu_13': 0.4,
#     'phi_1xdc': 0,
#     'phi_2xdc': 0,
#     'phi_3xdc': 0,
#     'name': 'conditional_tilt_xz'
# },
# {
#     'duration': 100,
#     'phi_1ax': 0.0,
#     'phi_1bx': 0.0,
#     'phi_2x': 0.0,
#     'phi_3x': 0.0,
#     'mu_12': 0.0,
#     'mu_13': 0.0,
#     'phi_1xdc': 0,
#     'phi_2xdc': 0,
#     'phi_3xdc': 0,
#     'name': '4 wells'
# }

]


# first initialization

### create initial state 

In [18]:
importlib.reload(coupled_fq_protocol_library)
default_init_state = False

if default_init_state:
    init_state = cfqr.init_state
else:
    init_state = None

In [19]:
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, protocol_list, initial_parameter_dict.keys())

# storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

In [20]:
importlib.reload(coupled_fq_protocol_library)

storage_protocol, comp_protocol = coupled_fq_protocol_library.create_system(computation_protocol_parameter_dict, initial_parameter_dict.keys(), modifiedFunction = None)

cfqr = cfq_3D_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity, measure_all_states = True)
cfqr.initialize_sim()
cfqr.set_sim_attributes(init_state = init_state)
cfqr.run_sim(init_state = init_state)

generating new initial_state
 found 500 samples out of 500
from system: finish the while loop in sampling.
as step value: slice(None, None, 10), sampleSize: 500
from cfq_runner.py, The as_step is slice(None, None, None) and dt is 0.0002
from quick_sim.py, sim_params:  [array([33.46640106, 33.46640106, 33.46640106, 33.46640106, 33.46640106,
       33.46640106]), array([1., 1., 1., 4., 4., 4.]), array([1.60216235, 1.60216235, 1.60216235, 3.20432471, 3.20432471,
       3.20432471])]
from quick_sim.py
gamma: [33.46640106 33.46640106 33.46640106 33.46640106 33.46640106 33.46640106], theta: [1. 1. 1. 4. 4. 4.] and eta: [1.60216235 1.60216235 1.60216235 3.20432471 3.20432471 3.20432471]
from quick_sim.py: system.protocol.t_i = 0.0, system.protocol.t_f = 200.0
from simulation.py: number of steps: 1000000, dt: 0.0002

 initializing...
generating new initial_state
 found 500 samples out of 500
from system: finish the while loop in sampling.
as step value: slice(None, None, 10), sampleSize: 500
f

In [21]:
init_state = cfqr.init_state
particle_index = cfqr.separate_by_state(init_state)
index_array = particle_index.values()
particle_color = ["#E69F00",  "#56B4E9",  "#009E73",  "#F5C710",  "#0072B2",  "#D55E00",  "#CC79A7", "#999999",  "#000000"]
all_states = cfqr.sim.output.all_state["states"]

In [22]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd

animation_data = all_states[:, 0::100, ...]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
title = ax.set_title('3D Test')


graph_array = []

for key, index, color in zip(particle_index.keys(), particle_index.values(), particle_color):
    xs = init_state[index][:, 0, 0]
    ys = init_state[index][:, 1, 0]
    zs = init_state[index][:, 2, 0]
    graph = ax.plot(xs, ys, zs, c = color, label = key, linestyle="", marker="o")
    graph_array.append(graph[0])
ax.legend(bbox_to_anchor=[-0.1, 0.5])

def update_graph(num):
    for index, _graph in zip(particle_index.values(), graph_array):
        x, y, z = animation_data[index, num, 0, 0], animation_data[index, num, 1, 0], animation_data[index, num, 2, 0]
        _graph.set_data (x, y)
        _graph.set_3d_properties(z)
        title.set_text('3D Test, time={}'.format(num))
    return title, _graph
# x, y, z = animation_data[..., 0, 0, 0], animation_data[..., 0, 1, 0], animation_data[..., 0, 2, 0]
# graph, = ax.plot(x, y, z, linestyle="", marker="o")

ani = matplotlib.animation.FuncAnimation(fig, update_graph, frames=animation_data.shape[1]-1, 
                               interval=50, blit=True)

plt.close()

In [23]:
html_ani = ani.to_html5_video()

In [24]:
HTML(html_ani)

In [ ]:
np.mean(cfqr.sim.work_dist_array)